# Continuous Control

### 1. Install dependencies
Most importantly install [Unity ML-agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md), PyTorch, and NumPy

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Make sure you have the Unity enviroment downloaded and change the path of the file_name

We are using the **20-agent environment** of Reacher.

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain brains which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Exame the State and Action Spaces
In this environment, a double-jointed arm can move to target locations. A reward of +0.1 is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of 33 variables corresponding to position, rotation, velocity, and angular velocities of the arm. Each action is a vector with four numbers, corresponding to torque applicable to two joints. Every entry in the action vector must be a number between -1 and 1.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

### 3. Instantiate and initialize the agent
The learning agent is imported from a separate file "./agent.py" and takes `state_size`, `action_size` and a `seed` as instance variables.

A few highlights of the agent:
- The agent selects the policy given by the actor-critic network
- The agent uses a buffer to store recent steps `(state, action, reward, next_state, done)` tuples and replay them
- The agent maximizes reward based on an actor-critic network

In [ ]:
from agent import Agent

BUFFER_SIZE = int(1e6)  # size of the memory buffer
BATCH_SIZE = 128        # sample minibatch size
GAMMA = 0.99            # discount rate for future rewards
LR_ACTOR = 1e-4         # learning rate of Actor
LR_CRITIC = 1e-4        # learning rate of Critic
TAU = 1e-3              # interpolation factor for soft update of target network
WEIGHT_DECAY = 0        # L2 weight decay
UPDATE_EVERY = 20       # update every 20 time steps
NUM_UPDATES = 10        # number of updates to the network

state_size = env_info.vector_observations.shape[1]  # size of the observation space (state space)
action_size = brain.vector_action_space_size        # size of the action space
num_agents = len(env_info.agents)                   # number of agents

agent = Agent(state_size, 
              action_size, 
              BUFFER_SIZE, 
              BATCH_SIZE, 
              num_agents,
              seed=0,
              gamma=GAMMA,
              tau=TAU,
              lr_actor=LR_ACTOR,
              lr_critic=LR_CRITIC,
              weight_decay=WEIGHT_DECAY,
              update_every=UPDATE_EVERY,
              num_updates=NUM_UPDATES)

### 4. Test randomly selected actions (untrained agent)
Run randomly selected actions in the environment to see what happens to the score. This is similar to an **untrained** agent.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step. A window should pop up that allows you to observe the agents, as it moves through the environment.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 5. Train multiple agents with Deep Determinitic Policy Gradient (DDPG)
The agent actually runs on an underlying actor-critic network. This is beneficial instead of using an typical deep Q-learning network (DQN) not only the environment's state space is large at 33 variables but the action space contains 4 continuous action variables. 

The setup is similar to the DQN with a local and target network; however, now there are separate networks to evaluate: the **actor** network for learning the optimal policy and the **critic** network for evaluating the selected action. 

Let's train the agents until they achieve an average score of +30 over 100 episodes.

In [ ]:
def ddpg(n_episodes=500, max_t=1000):
    '''
    -------------------------------------------
    Parameters
    
    n_episodes: # of episodes that the agent is training for
    max_t:      # of time steps (max) the agent is taking per episode
    -------------------------------------------
    '''
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.inf
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]            # turn on train mode of the environment
        states = env_info.vector_observations                        # get the current state for each agent
        agent.reset()                                                # reset the OU noise parameter 
        ep_scores = np.zeros(num_agents)                             # initialize the score for each agent
        for t in range(max_t):
            actions = agent.act(states)                              # select an action for each agent 
            env_info = env.step(actions)[brain_name]                 # send all actions to the environment
            next_states = env_info.vector_observations               # get next state for each agent
            rewards = env_info.rewards                               # get reward for each agent
            dones = env_info.local_done                              # check if episode finished
            agent.step(states, actions, rewards, next_states, dones) # agents record enviroment response in recent step
            states = next_states                                     # set the state as the next state for the following step for each agent
            ep_scores += rewards                                     # update the total score
            if np.any(dones):                                        # exit loop if episode for any agent finished
                break 
                
        scores_deque.append(np.mean(ep_scores))
        scores.append(ep_scores)
        
        # print average epsiode score and average 100-episode score for each episode
        print('\rEpisode {} \tScore: {:.2f} \tAverage Score: {:.2f}'.format(i_episode, np.mean(ep_scores), np.mean(scores_deque)), end="")  
        
        # print and save actor and critic weights when a score of +30 over 100 episodes has been achieved
        if np.mean(scores_deque) >= 30.0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_final.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_final.pth')
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break    
    return scores

scores = ddpg()

### Visualize the scores
Plot the scores according to their episodes. We can see a gradual increase in the scores as we increase the training episodes.

In [ ]:
fig = plt.figure()

# plot scores
plt.plot(np.arange(1, len(scores)+1), scores)
# plot average score/episode
plt.plot(np.arange(1, len(scores)+1), [np.mean(s_ep) for s_ep in scores], c='black', linewidth=2)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
env.close()

### 6. Test a trained agent
Run a **trained** agent for 1000 time steps to see what happens to the score. Compare this with the score of the untrained agent from 4.

In [ ]:
# Load policy network weights saved from training
agent.actor_local.load_state_dict(torch.load('checkpoint_actor_2.pth'))

env_info = env.reset(train_mode=False,)[brain_name] # reset enviroment and turn off training mode
states = env_info.vector_observations               # get first states from the resetted enviroment
scores = np.zeros(num_agents)                       

for t in range(1000):
    actions = agent.act(states, add_noise=False)     # select an action for each agent 
    env_info = env.step(actions)[brain_name]         # send all actions to the environment
    next_states = env_info.vector_observations       # get next state for each agent
    rewards = env_info.rewards                       # get reward for each agent
    dones = env_info.local_done                      # check if episode finished
    scores += rewards                                # set the state as the next state for the following step for each agent
    states = next_states                             # update the total score
    if np.any(dones):                                # exit loop if episode for any agent finished
        break
    
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()